<a href="https://colab.research.google.com/github/pragneshbarik/AI-ML/blob/main/otaku_recommender_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [4]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d marlesson/myanimelist-dataset-animes-profiles-reviews
! mkdir anime_dataset
! unzip /content/myanimelist-dataset-animes-profiles-reviews.zip -d anime_dataset

In [175]:
import pandas as pd
import numpy as np
import re
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [229]:
ps = PorterStemmer()
def punctuation_replacer(x) :
  to_replace = [':', "!", "?", "-", '"', ";", "+", "\r", "\n"]
  for c in to_replace :
    x = x.replace(c, "")
  
  return x

def stemmer(text) :
  stemmed_text = []
  for word in text.split() :
    stemmed_text.append(ps.stem(word))
  
  return " ".join(stemmed_text)

def recommend(uid) :
  idx = anime_df[anime_df['uid']==uid].index[0]
  args = (-similarity_matrix[idx]).argsort()[:1000]
  return args

In [230]:
anime_df = pd.read_csv("/content/anime_dataset/animes.csv")
anime_df = anime_df.drop_duplicates(subset=['uid']).dropna(subset=['synopsis', 'img_url']).fillna(0)
anime_df['id']=np.arange(len(anime_df['uid']))
anime_df.set_index('id', drop=True, inplace=True)

In [232]:
anime_df['synopsis']=anime_df['synopsis'].apply(punctuation_replacer).apply(lambda x:re.split(', |"|,|-| ' , x))
anime_df['genre']=anime_df['genre'].apply(lambda x: x.replace("'", "")).apply(lambda x:x[1:-1].split(',')).apply(lambda x:[i.replace(" ", '') for i in x])
anime_df['title_mod']  = anime_df['title'].apply(punctuation_replacer).apply(lambda x:x.split(" "))

In [ ]:
anime_df['tags'] = anime_df['genre']+anime_df['synopsis']
tagged_anime_df = anime_df[['uid', 'title', 'tags']]
tagged_anime_df['tags']=tagged_anime_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
vectorizer = CountVectorizer(max_features=5000, stop_words='english')
anime_vectors = vectorizer.fit_transform(tagged_anime_df['tags']).toarray()
tagged_anime_df['tags'] = tagged_anime_df['tags'].apply(stemmer)

In [236]:
similarity_matrix = cosine_similarity(anime_vectors)

In [311]:
anime_df['uid'].apply(recommend)
picks['uid'] = anime_df['uid']
picks['recommendations']= anime_df['uid'].apply(recommend)
picks.set_index('uid', drop=True, inplace=True)
picks.to_json('similarity_cache.json')
picks.to_json('similarity_cache.json')